# Create nfsw dataset

## Resolve depencencies

In [3]:
!pip install fake_useragent
!pip install imutils
!pip install gdown
!pip install kaggle --user
!pip install gdown
!pip install pyngrok flask
# !pip install tensorflowjsimport json

  Preparing metadata (setup.py) ... done
  Created wheel for fake_useragent: filename=fake_useragent-0.1.11-py3-none-any.whl size=13502 sha256=d6097715cc6242c410b977e761c01b99a67591aa29b4e0f67398f56c9ec1e485
  Stored in directory: /root/.cache/pip/wheels/ed/f7/62/50ab6c9a0b5567267ab76a9daa9d06315704209b2c5d032031
Successfully built fake_useragent
  Preparing metadata (setup.py) ... done
  Created wheel for imutils: filename=imutils-0.5.4-py3-none-any.whl size=25858 sha256=462c7a2403cf544abf84288ccf8e62051fabac89153ecded71d13b1fc61c7be2
  Stored in directory: /root/.cache/pip/wheels/86/d7/0a/4923351ed1cec5d5e24c1eaf8905567b02a0343b24aa873df2
Successfully built imutils
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for gdown: filename=gdown-4.5.1-py3-none-any.whl size=14933 sha256=53bb08aa76be5b4bb122d8e19817f69207c78e662fd27fc61c3d51b44eb7fb93
  Stored in directory: /root/.cache/pip/w

## Import main packages

In [4]:
import gdown
import zipfile
from pathlib import Path
from urllib.request import urlopen
import uuid
import requests
import os
import tensorflow as tf
import numpy as np
import cv2
import matplotlib.pyplot as plt
import hashlib
import sys
import random
import os
import tensorflow_hub as hub
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from fake_useragent import UserAgent
from matplotlib.widgets import Button
import datetime
from tensorflow.keras import layers
import pathlib
import logging
logger = tf.get_logger()
logger.setLevel(logging.ERROR)
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications import imagenet_utils
from imutils.object_detection import non_max_suppression
from PIL import Image
import scipy
import numpy as np
import argparse
import imutils
import time
import cv2
import requests
from IPython.display import clear_output
from io import BytesIO
from IPython.display import display, clear_output
#from ipywidgets import interact
#import ipywidgets as widgets
from PIL import ImageFilter
from bs4 import *
import uuid
from IPython.display import display, Markdown, clear_output, FileLink, FileLinks
from IPython.display import Image as IImage 
import ipywidgets as widgets
from PIL import Image
import pandas as pd
import gdown
from random import randrange
import ipywidgets as widgets
import shutil
from io import BytesIO
import json
import time
import mimetypes
from ipywidgets import GridspecLayout, Layout, Button, HBox, VBox, HTML, Image as IpywidgetsImage
import functools

### Kaggle api

In [5]:
credentials = {"username":"jamesdame","key":"157e2681cd07fe2460010b340d5a7504"}
kaggle_key_store='kaggle.json'
with open(kaggle_key_store, 'w', encoding='utf-8') as f:
    json.dump(credentials, f, ensure_ascii=False, indent=4)

!rm -rf /root/.kaggle     # when I created the folder, it says the file or dir already exits
!mkdir /root/.kaggle        # successful
!mv kaggle.json /root/.kaggle/kaggle.json    # not sure if I have to use full destination path, I previously only used /root/.kaggle and it failed. Don't have time to validate this thought.
!ls /root/.kaggle/kaggle.json
!chmod 600 /root/.kaggle/kaggle.json

/root/.kaggle/kaggle.json


## Declare global variables

In [6]:
dwd_url = "https://drive.google.com/uc?id=1h1CeZ_tU5FhuSm4EvkLk1zqbHUOgPMuV"
dwd_file_name = "nsfw-content-moderation-data.zip"
dwd_dir = dwd_file_name.replace(".zip", "")
# paths = os.listdir(dwd_dir+"/raw_data")
DATA_CLASSES = ["sexy", "porn", "hentai", "neutral"]
DATA_ROOT_PATH = "./data"
RAW_DATA_DIR = os.path.join(dwd_dir, "raw_data")
DATA_DIR = os.path.join(DATA_ROOT_PATH, "images")
DATASET_PATH = DATA_DIR
DATASET_DUMP_PATH = "./dump"
IMAGE_RES = 224
CLEAN = True
CSV_DATASET_PATH = os.path.join(DATA_ROOT_PATH, "data.csv")
DATASET_NAME = "nsfw-content-moderation"

## Create main functions

In [7]:
dump_dir = DATASET_DUMP_PATH
def preview_images_from_directory(path=DATASET_PATH, group=True):
    dimensions=(IMAGE_RES, IMAGE_RES)
    data_dir = path
    clean_up_data_dir(data_dir)
    images_path = []
    
    if(group):
        data_sub_directories = os.listdir(data_dir)
        for data_sub_directory in data_sub_directories:
#             print("found {0} for class {1}".format(len(os.listdir(os.path.join(data_dir, data_sub_directory))), data_sub_directory))
            for current_dir in os.listdir(os.path.join(data_dir, data_sub_directory)):
                images_path.append(os.path.join(data_sub_directory, current_dir))
    else:
        try:
            for current_dir in os.listdir(data_dir):
                images_path.append(os.path.join(data_dir, current_dir))
        except Exception as wrong:
            pass

    if not group:
        data_dir = "."
        
    current = 0        
    output = widgets.Output()
    next_button = widgets.Button(description='Next')
    prev_button = widgets.Button(description='Prev')
    display_current_button = widgets.Button(description='Current')
    current_index_text = widgets.Textarea(
        value=str(current),
        placeholder='current index goes here',
        description='index',
        disabled=False
    )
    
    display(current_index_text, display_current_button, prev_button, next_button, output)
    
    def default_action():
        global current
        with output:
            clear_output()
            print("{0}: {1}/{2}".format(images_path[current].split("/")[0], current+1, len(images_path)))
            image = cv2.imread(os.path.join(data_dir, images_path[current]))
            # imageBGR = cv2.imdecode(image, cv2.IMREAD_COLOR)
            imageRGB = cv2.cvtColor(image , cv2.COLOR_BGR2RGB)
            image_resized = cv2.resize(imageRGB, dimensions, interpolation = cv2.INTER_AREA)/255
            
            current_image = Image.fromarray(cv2.resize(imageRGB, dimensions, interpolation = cv2.INTER_AREA))
            display(current_image)
            
    def on_next_button_clicked(_):
        global current
        if current+2 > len(images_path):
            return None
        current+=1
        default_action()


    def on_prev_button_clicked(_):
        global current
        if current-1 < 0:
            return None
        current-=1
        default_action()
        
        
    def on_current_index_change(_):
        update_index_change(current_index_text.value)

    def update_index_change(indexString):
        global current
        try:
            current = int(indexString)
            default_action()
        except Exception as wrong:
            pass

    next_button.on_click(on_next_button_clicked)
    prev_button.on_click(on_prev_button_clicked)
    display_current_button.on_click(on_current_index_change)
    current_index_text.on_displayed(update_index_change(str(current)))
    
    
    
    
def order_images(main_dir, start=-1, end=-1, figsize=(30, 30), dimensions=(IMAGE_RES, IMAGE_RES)):
    %matplotlib inline
    from IPython.display import display, Markdown, clear_output
    from IPython.display import Image as IImage 
    import ipywidgets as widgets

    current  =  0
    del_dir = os.path.join(main_dir, ".ipynb_checkpoints")
    !rm -r $del_dir
    if start == -1 and end == -1:
        images_path = os.listdir(main_dir)
    elif start != -1 and end == -1:
        images_path = os.listdir(main_dir)[start:len(os.listdir(main_dir))]
    elif start == -1 and end != -1:
        images_path = os.listdir(main_dir)[0:end]
    else:
        images_path = os.listdir(main_dir)
        
    next_button = widgets.Button(description='Next')
    prev_button = widgets.Button(description='Prev')
    move_button = widgets.Button(description='Move')
    class_names = os.listdir(DATASET_PATH)
    moving_paths = []

    path_selector = widgets.SelectMultiple(
        options=class_names,
        value=[],
        description='Fruits',
        disabled=False
    )
    output = widgets.Output()
    display(prev_button, next_button, output, path_selector, move_button)

    def on_next_button_clicked(_):
        global current
        if current+2 > len(images_path):
            return None
        moving_paths = []
        with output:
            current+=1
            clear_output()
            print("{0}/{1}".format(current+1, len(images_path)))
            pil_img = IImage(filename=os.path.join(main_dir, images_path[current]), width = dimensions[0], height=dimensions[1])
            display(pil_img)

    def on_prev_button_clicked(_):
        global current
        if current-1 < 0:
            return None
        moving_paths = []
        with output:
            current-=1
            clear_output()
            print("{0}/{1}".format(current+1, len(images_path)))
            pil_img = IImage(filename=os.path.join(main_dir, images_path[current]), width = dimensions[0], height=dimensions[1])
            display(pil_img)
    def on_move_button_clicked(_):
        with output:
            print(path_selector.value)
            for current_path in path_selector.value:
                os.system("cp '{0}' '{1}'".format(os.path.join(main_dir, DATASET_PATH+"/"+images_path[current]), current_path))

    next_button.on_click(on_next_button_clicked)
    prev_button.on_click(on_prev_button_clicked)
    move_button.on_click(on_move_button_clicked)

def clean_up_data_dir(data_dir):
    data_sub_directories = os.listdir(data_dir)
    for data_sub_directory in data_sub_directories:
        path_to_delete = os.path.join(data_dir, data_sub_directory, ".*")
        !rm -r $path_to_delete

    !rm -r $data_dir/.ipynb_checkpoints
    !rm -r $data_dir/.DS_Store

# CREATE FOLDER
def folder_create(images, given_folder_name=""):
    try:
        folder_name = os.path.join(dump_dir, input("Enter Folder Name:- ") if given_folder_name == "" else given_folder_name)
        # folder creation
        os.mkdir(folder_name)
 
    # if folder exists with that name, ask another name
    except:
        print("Folder Exist with that name!")
        folder_create()
 
    # image downloading start
    download_images(images, folder_name)
 
 
# DOWNLOAD ALL IMAGES FROM THAT URL
def download_images(images, folder_name):
   
    # initial count is zero
    count = 0
 
    # print total images found in URL
    print(f"Total {len(images)} Image Found!")
 
    # checking if images is not zero
    if len(images) != 0:
        for i, image in enumerate(images):
            # From image tag ,Fetch image Source URL
 
                        # 1.data-srcset
                        # 2.data-src
                        # 3.data-fallback-src
                        # 4.src
 
            # Here we will use exception handling
 
            # first we will search for "data-srcset" in img tag
            try:
                # In image tag ,searching for "data-srcset"
                image_link = image["src"]
                 
            # then we will search for "data-src" in img
            # tag and so on..
            except:
                try:
                    # In image tag ,searching for "data-src"
                    image_link = image["data-src"]
                except:
                    try:
                        # In image tag ,searching for "data-fallback-src"
                        image_link = image["data-fallback-src"]
                    except:
                        try:
                            # In image tag ,searching for "src"
                            image_link = image["data-srcset"]
 
                        # if no Source URL found
                        except:
                            pass
 
            # After getting Image Source URL
            # We will try to get the content of image
            try:
                print("Downloading image: {0}/{1}; store => {2}".format(count, len(images), folder_name))
                r = requests.get(image_link).content
                try:
 
                    # possibility of decode
                    r = str(r, 'utf-8')
                except UnicodeDecodeError:
 
                    # After checking above condition, Image Download start
                    with open(f"{folder_name}/images{i+1}.jpg", "wb+") as f:
                        f.write(r)
 
                    # counting number of image downloaded
                    count += 1
            except:
                pass
 
        # There might be possible, that all
        # images not download
        # if all images download
        if count == len(images):
            print("All Images Downloaded!")
             
        # if all images not download
        else:
            print(f"Total {count} Images Downloaded Out of {len(images)}")
 
# MAIN FUNCTION START
def download_images_from_url(url, given_folder_name=""):
    if url == None or ( url.find("http") == -1 and url.find("www") == -1 ):
        return
    # content of URL
    ua = UserAgent()

    # Get list of user agents.


    # headers = {'User-Agent': ua.random}
    # r = requests.get(url, headers=headers)
    r = requests.get(url)
    print(url)
    print(r)
    # Parse HTML Code
    soup = BeautifulSoup(r.text, 'html.parser')
 
    # find all images in URL
    images = soup.findAll('img')
 
    # Call folder create function
    folder_create(images, given_folder_name)

def remove_duplicates(dir, include_src=False):
    hashMap = {}
    # List to store deleted files
    deletedFiles = []
    source_dup_file = []
    filelist = os.listdir(dir)
    for f in filelist:
        f = os.path.join(dir, f)
        key = hashFile(f)
        # If key already exists, it deletes the file
        if key in hashMap.keys():
            deletedFiles.append(f)
            if include_src:
                try:
                    index = source_dup_file.index(key)
                except Exception as e:
                    source_dup_file.append(key)
            os.remove(f)
        else:
            hashMap[key] = f
    if include_src:
        for key in source_dup_file:
            deletedFiles.append(f)
            os.remove(hashMap[key])
            
    if len(deletedFiles) != 0:  
        for deleted_file in deletedFiles:
            print('Deleted Files {0}'.format(deleted_file))
        print("total deleted => {}".format(len(deletedFiles)))
    else:
        print('No duplicate files found')
    

def remove_small_files(dir, min_size=5):
    for root, _, files in os.walk(dir):
        for f in files:
            fullpath = os.path.join(root, f)
            try:
                if os.path.getsize(fullpath) < min_size * 1024:   #set file size in kb
                    print(fullpath)
                    os.remove(fullpath)
            except Exception as e:
                print("Error" + fullpath)

def rename_all_files(dir):
    for root, _, files in os.walk(dir):
        for f in files:
            fullpath = os.path.join(root, f)
            try:
                filename, file_extension = os.path.splitext(fullpath)
                newname = str(uuid.uuid1())+"."+file_extension
                os.rename(fullpath, os.path.join(dir, newname))
           
            except Exception as e:
                print(e)
                print("Error" + fullpath)
                
def delete_unreadable_images(dir):
    for root, _, files in os.walk(dir):
        for f in files:
            fullpath = os.path.join(root, f)
            try:
                img = Image.open(fullpath)
            except Exception as e:
                os.system("rm {}".format(fullpath))
                print("Removing => " + fullpath)

def remove_randomly_dir_files(dir, limit=2, percentage=0):
    files = os.listdir(dir)

    if(percentage != 0):
        limit = int((percentage * len(files)) / 100)

    print("Total files found : {}".format(len(files)))
    deleted_indexes = [-1]
    count = 0
    if limit >= len(files):
        print("limit >= len(files)")
        return 

    for i in range(0, limit):
        if len(deleted_indexes) > limit:
            print("len(deleted_index) > limit")
            break

        random_index = -1

        while ( random_index in deleted_indexes) == True:
            random_index = random.randint(0, len(files)-1)

        deleted_indexes.append(random_index)
        count+=1

        print("deleting {0}/{1}; index => {2}".format(i+1, limit, random_index))
        os.remove(os.path.join(dir, files[random_index]))

    print("Total deleted files {}".format(count))
    print("Total files remaining {}".format(len(os.listdir(dir))))
    
def hashFile(filename):
    # For large files, if we read it all together it can lead to memory overflow, So we take a blocksize to read at a time
    BLOCKSIZE = 65536
    hasher = hashlib.md5()
    with open(filename, 'rb') as file:
        # Reads the particular blocksize from file
        buf = file.read(BLOCKSIZE)
        while(len(buf) > 0):
            hasher.update(buf)
            buf = file.read(BLOCKSIZE)
    return hasher.hexdigest()

def predict_at_random_download(base_url="https://picsum.photos/{0}/{0}".format(IMAGE_RES), store=DATASET_PATH, group=True):
    selected_class = class_names[0]
    selected_image = []
    again_button = widgets.Button(description='Again')
    download_button = widgets.Button(description='Download')
    class_selector = widgets.Dropdown(
        options=class_names,
        value=selected_class,
        description='Select a class',
        disabled=False,
    )

    output = widgets.Output()
    display(again_button, class_selector, download_button, output)

    def on_again_button_clicked(_):
        global selected_image
        with output:
            clear_output()
            to_print, image = predict_single_image_from_url(base_url)
            selected_image = image
            print(to_print)
            display(image)
            
    def on_download_button_clicked(_):
        global selected_image
        global selected_class
        with output:
            try:
                if group:  
                    export_path = "{}.jpg".format(os.path.join(store, selected_class, str(uuid.uuid1())))
                else:
                    export_path = "{}.jpg".format(os.path.join(store, str(uuid.uuid1())))
                print("selected class => ", selected_class)
                print("export path => ", export_path)
                selected_image.save(export_path)
                local_file = FileLink(export_path, result_html_prefix="Click here to download model: ")
                display(local_file)
            except Exception as wrong:
                print("error while moving file to =>", selected_class)
                print(wrong)
    def on_class_change(change):
        global selected_class
        try:
            selected_class = class_names[change["new"]["index"]]
        except Exception as wrong:
            pass
        

    with output:
        clear_output()
        to_print, image = predict_single_image_from_url(base_url)
        selected_image = image
        print(to_print)
        display(image)
    
    again_button.on_click(on_again_button_clicked)
    download_button.on_click(on_download_button_clicked)
    class_selector.observe(on_class_change)
    
def download_collection_from_unsplash(collection_id = "500522", selected_class="", store="", group=True, perPage=10, page=1, image_width=IMAGE_RES, image_height=IMAGE_RES, fit="crop", download_link=False):
    api_key = '3E1O5xqWI-Opz3W81XdmIvZwPJ2qFTHggE5YUxZysDg'
    url = "https://api.unsplash.com/collections/{0}/photos?page={1}&per_page={2}".format(collection_id, page, perPage)
    # ua = UserAgent()
    # headers = {'User-Agent': ua.random, 'Authorization': "Client-ID {}".format(api_key)}
    headers = {
        'Authorization': "Client-ID {}".format(api_key),
        }
    images = []
    count = 0
    total = 0
    total_pages = 1
    current_page = page
    
    try:
        response = requests.get(url.format(collection_id, current_page, perPage), headers=headers)
        total = int(response.headers['X-Total'])
        
        while(total_pages * perPage < total):
            total_pages+=1
    except Exception as wrong:
        print("count error")
        print(wrong)
        
    while current_page <= total_pages:
        print("processing page ", current_page, "/", total_pages)
        try:
            response = requests.get(url.format(collection_id, current_page, perPage), headers=headers)
            response_json = response.json()
            for image_data in response_json:
                print("downloading image ",count+1, "/", total)
                try:
                    if group:  
                        export_path = "{}.jpg".format(os.path.join(store, selected_class, str(uuid.uuid1())))
                    else:
                        export_path = "{}.jpg".format(os.path.join(store, str(uuid.uuid1())))
                    image_response = requests.get(image_data["urls"]["raw"]+"&w={0}&h={1}&fit={2}".format(image_width, image_height, fit))
                    #raw,full, regular, small, thumb, small_s3    
                    image = np.asarray(bytearray(image_response.content), dtype="uint8")
                    imageBGR = cv2.imdecode(image, cv2.IMREAD_COLOR)
                    imageRGB = cv2.cvtColor(imageBGR , cv2.COLOR_BGR2RGB)
                    cv2.imwrite(export_path, imageBGR)
                    if download_link:
                        local_file = FileLink(export_path, result_html_prefix="Click here to download: ")
                        display(local_file)            
                except Exception as wrong:
                    print(wrong)
                    print("error while downloading image")
                count+=1
        except Exception as wrong:
            print("error while loading collections photos")
        current_page+=1
        
def dwd_images(destination, source_file="*.txt", limitPoint = -1, count_set=0):
    total = len(open(Path(source_file)).readlines())
    image_input_file = open(source_file, "r")
    count = count_set
    image_input_file = [image_input_fileS for image_input_fileS in image_input_file]
    ids = str(uuid.uuid1())
    
    if limitPoint != -1 and len(image_input_file) > limitPoint:
        image_url_list = image_input_file[count_set:limitPoint]
        
    for url in image_url_list:
        count+=1
        try:
            currentId = str(uuid.uuid1())
            file = destination+"/"+ids+currentId+".jpg"
            response = urlopen(url, timeout = 3)
            with open(file, 'wb') as f:
                f.write(response.read())
            f.close()
            
            print("{0} / {1} success => {2} path => {3}".format(count, total, url, file))

        except Exception as wrong: 
            print("{0} / {1} req failure => {2} : {3}".format(count, total, url, wrong))
            pass
        
        
        
def image_gallery_from_directory(path=DATASET_PATH, group=True, default_page=0):
    dimensions=(IMAGE_RES, IMAGE_RES)
    data_dir = path
#     clean_up_data_dir(data_dir)
    images_path = []
    if(group):
        data_sub_directories = os.listdir(data_dir)
        for data_sub_directory in data_sub_directories:
#             print("found {0} for class {1}".format(len(os.listdir(os.path.join(data_dir, data_sub_directory))), data_sub_directory))
            for current_dir in os.listdir(os.path.join(data_dir, data_sub_directory)):
                images_path.append(os.path.join(data_sub_directory, current_dir))
    else:
        try:
            for current_dir in os.listdir(data_dir):
                images_path.append(os.path.join(data_dir, current_dir))
        except Exception as wrong:
            pass

    if not group:
        data_dir = "."
        
    current = default_page
    output = widgets.Output()
    next_button = widgets.Button(description='Next')
    prev_button = widgets.Button(description='Prev')
    display_current_button = widgets.Button(description='Current')
    current_index_text = widgets.Textarea(
        value=str(current),
        placeholder='current index goes here',
        description='index',
        disabled=False
    )
    
    delete_button = widgets.Button(description='Delete')
    
    display(current_index_text, display_current_button, prev_button, next_button, output)
    
    deleted_images = []
    
    def get_button(description):
        return widgets.Button(description=description, style=dict(
                                            font_style='italic',
                                            font_weight='bold',
                                            font_variant="small-caps",
                                            text_color='red',
                                            text_decoration='underline',
                                            font_size=20,
#                                             width=100
                    ))
    items_per_page = 24
    num_rows = 4
    total_pages = int(len(images_path)/items_per_page)
   
        
    def default_action():

        global current
#         global images_path
        with output:
            clear_output()
            print("{0}: {1}/{2}".format(images_path[current*items_per_page].split("/")[0], current, total_pages))
            num_images = 0
            images_line_1 =  []
            buttons_line_1 =  []
            images_line_2 =  []
            buttons_line_2 =  []
            images_line_3 =  []
            buttons_line_3 =  []
            images_line_4 =  []
            buttons_line_4 =  []
            while num_images < items_per_page:
                try:
                    image_path = os.path.join(data_dir, images_path[current*items_per_page+num_images])
                    cat = images_path[current*items_per_page+num_images].split("/")[0]
                    file = open(image_path, "rb")
                    image = file.read()
                    image = IpywidgetsImage(
                        value=image,
                        format='jpeg',
                        width=160,
                        height=140,
                    )
                    
                    def on_current_delete_button_click(_, path=""):
                        os.remove(path)
                        default_action()

                    button = get_button("Delete {0} {1} forever".format(cat, num_images))
                    button.on_click(functools.partial(on_current_delete_button_click, path=image_path))
                    

                    if num_images < items_per_page/num_rows:
                        images_line_1.append(image)
                        buttons_line_1.append(button)
                    elif num_images < items_per_page/num_rows*2:
                        images_line_2.append(image)
                        buttons_line_2.append(button)
                    elif num_images < items_per_page/num_rows*3:
                        images_line_3.append(image)
                        buttons_line_3.append(button)
                    else:
                        images_line_4.append(image)
                        buttons_line_4.append(button)
                        
                except Exception as wrong:
#                     print(wrong)
                    pass
                num_images+=1
                
            display(HBox(images_line_1))
            display(HBox(buttons_line_1))
            display(HBox(images_line_2))
            display(HBox(buttons_line_2))
            display(HBox(images_line_3))
            display(HBox(buttons_line_3))
            display(HBox(images_line_4))
            display(HBox(buttons_line_4))
            display(current_index_text, display_current_button, prev_button, next_button, output)
            
    def on_next_button_clicked(_):
        global current
        if current+2 > len(images_path):
            return None
        current+=1
        default_action()


    def on_prev_button_clicked(_):
        global current
        if current-1 < 0:
            return None
        current-=1
        default_action()
        
        
    def on_current_index_change(_):
        update_index_change(current_index_text.value)

    def update_index_change(indexString):
        global current
        try:
            current = int(indexString)
            default_action()
        except Exception as wrong:
            pass

    next_button.on_click(on_next_button_clicked)
    prev_button.on_click(on_prev_button_clicked)
    display_current_button.on_click(on_current_index_change)
    current_index_text.on_displayed(update_index_change(str(current)))

## Download dataset

In [8]:
gdown.download(dwd_url, dwd_file_name, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1h1CeZ_tU5FhuSm4EvkLk1zqbHUOgPMuV
To: /kaggle/working/nsfw-content-moderation-data.zip
100%|██████████| 59.9M/59.9M [00:00<00:00, 221MB/s]


'nsfw-content-moderation-data.zip'

In [9]:
with zipfile.ZipFile(dwd_file_name,"r") as zip_ref:
    zip_ref.extractall("./")

##  Get current dataset version

In [10]:
!cp -a /kaggle/input/nsfw-content-moderation/. $DATA_ROOT_PATH


## Create main directories

In [11]:
!mkdir $DATA_ROOT_PATH
!mkdir $DATA_DIR
!touch $CSV_DATASET_PATH

mkdir: cannot create directory ‘./data’: File exists
mkdir: cannot create directory ‘./data/images’: File exists


## See available classes

In [12]:
!ls $dwd_dir/raw_data

 age_college
 age_mature
 age_milf
 age_teen
 amateur
 amateur_self-shots
 appearance
 appearance_appearance-modification
 appearance_appearance-modification_piercings
 appearance_appearance-modification_tattoos
 appearance_clothing
 appearance_clothing_bodyparts-through-clothes
 appearance_clothing_bottomless
 appearance_clothing_clothed-naked-pair
 appearance_clothing_dresses
 appearance_clothing_shoes
 appearance_clothing_stockings
 appearance_clothing_swimwear
 appearance_clothing_tight-clothing
 appearance_clothing_topless
 appearance_clothing_underwear
 appearance_clothing_underwear_panties
 appearance_clothing_underwear_thongs
 appearance_clothing_uniforms-outfits
 appearance_clothing_uniforms-outfits_cosplay
 appearance_clothing_upskirt-downblouse
 appearance_expressions
 appearance_pose
'appearance_wet-&-messy'
 artificial-images
 artificial-images_fictional-characters-shows
 artificial-images_hentai
 artificial-images_my-little-pony
 artificial-images_photoshop
 body-parts_he

In [13]:
cat = "neutral"
path = "{0}/raw_data/{1}/urls_{1}.txt".format(dwd_dir, cat)
!wc -l $path

36837 nsfw-content-moderation-data/raw_data/neutral/urls_neutral.txt


## Initialize dataset creation

In [ ]:
for path in DATA_CLASSES:
    try:
        data_path = os.path.join(dwd_dir, "raw_data", path)
        store_path = os.path.join(DATA_DIR, path)
        !mkdir $store_path
        if CLEAN:
            !rm -r $store_path/* 
        files = [f for f in os.listdir(data_path) if os.path.isfile(os.path.join(data_path, f))]
        dwd_images(store_path, source_file=os.path.join(data_path, files[0]), count_set=2002, limitPoint=36837)
    except Exception as wrong: 
        print(wrong)

## Delete corrupted files

In [ ]:
data_sub_directories = os.listdir(DATASET_PATH)
for data_sub_directory in data_sub_directories:   
    delete_unreadable_images(os.path.join(DATASET_PATH, data_sub_directory))

## Remove duplicates

In [ ]:
data_sub_directories = os.listdir(DATASET_PATH)
for data_sub_directory in data_sub_directories:   
    remove_duplicates(os.path.join(DATASET_PATH, data_sub_directory))

## Remove small files

In [ ]:
data_sub_directories = os.listdir(DATASET_PATH)
for data_sub_directory in data_sub_directories:   
    remove_small_files(os.path.join(DATASET_PATH, data_sub_directory))

## Edit data relationship

In [ ]:
classes_meta = {
#     "female_nudity": {
#         "childs":["general_nsfw"]
#     },
    
#     "general_nsfw":{
#         "childs": ["female_nudity"]
#     },
    
#     "female_underwear":{
#         "childs": ["male_underwear"]
#     },
#     "male_underwear":{
#         "childs": ["female_underwear"]
#     }
}


data_dir = DATASET_PATH
data = []
data_sub_directories = os.listdir(data_dir)
for data_sub_directory in data_sub_directories:
    files = os.listdir(os.path.join(data_dir, data_sub_directory))
    for file in files:
        file_meta = {}
        file_meta["filenames"]=os.path.join(data_sub_directory, file)
        
        class_childs = []
        if data_sub_directory in classes_meta and USE_DATA_RELATIONSHIP:
            class_childs = classes_meta[data_sub_directory]["childs"]

        for current_class_name in data_sub_directories:
            if current_class_name == data_sub_directory or current_class_name in class_childs:
                file_meta[current_class_name] = str(1).replace(".0", "")                        
            else:
                file_meta[current_class_name] = str(0)
        data.append(file_meta)

df = pd.DataFrame(data)
df.to_csv(CSV_DATASET_PATH, encoding='utf-8', index=False)

print("done => ", len(data))
df.describe()

## Preview Dataset

In [1]:
preview_images_from_directory(DATASET_PATH)

NameError: name 'preview_images_from_directory' is not defined

## updating kaggle dataset version 

In [ ]:
!kaggle datasets status $DATASET_NAME
!kaggle datasets metadata -p $DATA_ROOT_PATH $DATASET_NAME
save_path = 'updated_data_{}'.format(time.time())
!kaggle datasets version -p $DATA_ROOT_PATH -m $save_path --dir-mode zip

### Download images from server

In [17]:
domain = "https://mgx-api.karamokoisrael.tech"

In [18]:
req = requests.get(domain+"/items/feedbacks?filter[moderation_classes][_nnull]=true&filter[downloaded_for_dataset][_eq]=false&filter[image][_nnull]=true")
data = req.json()["data"]

In [19]:
count = 0
total = len(data)
for fileData in data: 
    try:
        file =  "data/images/{0}/hp-{1}.jpg" .format(fileData["moderation_classes"][0], fileData["image"])
        print("processing {0}/{1} => {2}".format(count+1, total, file))
        dwd_url = "{0}/file/{1}".format(domain, fileData["image"])
        image_response = requests.get(dwd_url)  
        image = np.asarray(bytearray(image_response.content), dtype="uint8")
        imageBGR = cv2.imdecode(image, cv2.IMREAD_COLOR)
        imageRGB = cv2.cvtColor(imageBGR , cv2.COLOR_BGR2RGB)
        cv2.imwrite(file, imageBGR)            
        local_file = FileLink(file, result_html_prefix="Download {0}/{1}".format(count+1, total))
        display(local_file)  
        count+=1
    except Exception as wrong:
        print("error for {0}/{1} => {2}".format(count+1, total, wrong))
        count+=1
        pass

processing 1/1 => data/images/neutral/hp-8794f052-6c2f-43df-b943-7bb757b267d3.jpg


/kaggle/working/data/images/neutral/hp-8794f052-6c2f-43df-b943-7bb757b267d3.jpg

## File Manager

In [18]:
!jupyter labextension list
!pip uninstall jupyterlab_widgets -y
!pip install jupyterlab_widgets

JupyterLab v3.2.9
/opt/conda/share/jupyter/labextensions
        nbdime-jupyterlab v2.1.1 enabled OK
        jupyterlab-jupytext v1.3.8+dev enabled OK (python, jupytext)
        jupyterlab_pygments v0.2.2 enabled OK (python, jupyterlab_pygments)
        jupyter-threejs v2.4.0 enabled OK (python, pythreejs)
        bqplot v0.5.37 enabled OK (python, bqplot)
        jupyter-vuetify v1.8.4 enabled OK
        jupyter-vue v1.7.0 enabled OK
        jupyter-matplotlib v0.9.0 enabled OK
        jupyterlab-datawidgets v7.1.2 enabled OK
        jupyter-webrtc v0.6.0 enabled OK
        jupyter-leaflet v0.17.1 enabled OK
        catboost-widget v1.0.0 enabled OK
        jupyterlab-plotly v5.10.0 enabled OK
        @jupyter-widgets/jupyterlab-manager v3.1.1 enabled OK (python, jupyterlab_widgets)
        @jupyterlab/server-proxy v3.2.1 enabled OK
        @jupyterlab/git v0.37.1 enabled OK (python, jupyterlab-git)
        @krassowski/jupyterlab-lsp v3.10.2 enabled OK (python, jupyterlab-lsp)
       

In [23]:
image_gallery_from_directory(default_page=53)

hentai: 53/279


Textarea(value='53', description='index', placeholder='current index goes here')

Button(description='Current', style=ButtonStyle())

Button(description='Prev', style=ButtonStyle())

Button(description='Next', style=ButtonStyle())

Output(msg_id='9d2f3e7e-33c9-4382-a74c-a6cb36160f1d')

## updating kaggle dataset version 

In [ ]:
!kaggle datasets status $DATASET_NAME
!kaggle datasets metadata -p $DATA_ROOT_PATH $DATASET_NAME
save_path = 'updated_data_{}'.format(time.time())
!kaggle datasets version -p $DATA_ROOT_PATH -m $save_path --dir-mode zip